##자연어 추론

In [6]:
!pip install transformers datasets accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [3]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
"klue/bert-base", num_labels=3, id2label={0: '함의', 1: '중립', 2: '모순'})
model.to('cuda:0')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [4]:
from datasets import load_dataset
train_ds_raw = load_dataset('klue', 'nli', split='train')
eval_ds_raw = load_dataset('klue', 'nli', split='validation')

Generating train split:   0%|          | 0/24998 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [6]:
def convert_example(example):
  result = tokenizer(example['premise'], example['hypothesis'])
  result['label'] = [example['label']]
  return result
train_ds = train_ds_raw.map(convert_example)
eval_ds = eval_ds_raw.map(convert_example)

Map:   0%|          | 0/24998 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [8]:
import numpy as np
import evaluate
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [9]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
#트레이닝 설정
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
output_dir="test_trainer",
num_train_epochs=1,
evaluation_strategy="epoch")

In [11]:
trainer = Trainer(
model=model,
args=training_args,
train_dataset=train_ds,
eval_dataset=eval_ds,
data_collator=data_collator,
compute_metrics=compute_metrics,
)
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.495100,0.536165,0.798333


TrainOutput(global_step=3125, training_loss=0.5816420446777344, metrics={'train_runtime': 454.8335, 'train_samples_per_second': 54.961, 'train_steps_per_second': 6.871, 'total_flos': 773319112732656.0, 'train_loss': 0.5816420446777344, 'epoch': 1.0})

In [12]:
#자연어 추론을 위한 파이프라인이 없으므로 텍스트 분류 파이프라인을 그대로 사용
from transformers import pipeline
nli = pipeline(task='text-classification', model=model,
tokenizer=tokenizer, device='cuda:0')

In [13]:
# 두 문장 사이에 [SEP] 토큰을 넣어, 구분만 해주면 됨
nli('아침을 안 먹었다[SEP]배가 고프다')

[{'label': '중립', 'score': 0.49789637327194214}]

In [14]:
nli('차를 놓쳤다[SEP]제 시간에 도착했다')

[{'label': '모순', 'score': 0.9046080708503723}]

## 유사도

In [4]:
# 유사도는 분류와 회귀 모두 사용할 수 있으나 여기서는 회귀로 처리
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
"klue/bert-base",
num_labels=1, problem_type='regression')
model.to('cuda:0');

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# 토크나이저는 자연어 추론 실습과 같음
# 데이터셋
from datasets import load_dataset
sts_train = load_dataset('klue', 'sts', split='train')
sts_eval = load_dataset('klue', 'sts', split='validation')

Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]

In [6]:
#전처리 함수
def convert_example(example):
  result = tokenizer(example['sentence1'], example['sentence2'])
  result['label'] = example['labels']['real-label']
  return result

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [10]:
# 전처리 함수를 일괄 적용하고 기존 데이터의 labels 컬럼은 삭제
train_ds = sts_train.map(convert_example, remove_columns=['labels'])
eval_ds = sts_eval.map(convert_example, remove_columns=['labels'])

Map:   0%|          | 0/11668 [00:00<?, ? examples/s]

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

In [14]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
#트레이닝 설정
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
output_dir="test_trainer",
num_train_epochs=1,
evaluation_strategy="epoch")

In [15]:
# training_args와 data_collator는 자연어 추론 실습과 같음
trainer = Trainer(
model=model,
args=training_args,
train_dataset=train_ds,
eval_dataset=eval_ds,
data_collator=data_collator)
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.223400,0.554823


TrainOutput(global_step=1459, training_loss=0.27209490958430166, metrics={'train_runtime': 224.7121, 'train_samples_per_second': 51.924, 'train_steps_per_second': 6.493, 'total_flos': 399766937024712.0, 'train_loss': 0.27209490958430166, 'epoch': 1.0})

In [16]:
# 회귀의 경우에는 파이프라인이 없으므로 수동 처리
# 토큰화
enc = tokenizer([
'방안에 필요한 시설이 모두 있음[SEP]숙소 근처에 안전한 실내 주차장이 있음',
'호스트의 답장이 늦음[SEP]호스트 응답이 늦음',
], padding=True, truncation=True, return_tensors='pt').to('cuda:0')

In [17]:
# 모델 적용
model(**enc).logits

tensor([[0.5890],
        [0.9444]], device='cuda:0', grad_fn=<AddmmBackward0>)

## 문장 임베딩

In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4bac03f3527ce84017e1f9716fc67f082447af7d51485c8f5215663adbca4114
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [2]:
from sentence_transformers import InputExample
import tqdm.auto as tqdm
def convert_example(dataset):
  pairs = []
  for example in tqdm.tqdm(dataset):
    pairs.append(InputExample(
      texts=[example['sentence1'], example['sentence2']],
      label=example['labels']['binary-label']))
  return pairs

In [7]:
# 데이터셋
from datasets import load_dataset
sts_train = load_dataset('klue', 'sts', split='train')
sts_eval = load_dataset('klue', 'sts', split='validation')

Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]

In [8]:
# 전처리
train_examples = convert_example(sts_train)
eval_examples = convert_example(sts_eval)

  0%|          | 0/11668 [00:00<?, ?it/s]

  0%|          | 0/519 [00:00<?, ?it/s]

In [9]:
# 데이터 로딩
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=2)

In [10]:
from sentence_transformers import SentenceTransformer
from sentence_transformers .models import Transformer, Pooling
word_emb_model = Transformer('klue/bert-base', max_seq_length=256)
pooling_model = Pooling(word_emb_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_emb_model, pooling_model])

In [11]:
# 데이터에서 의미가 비슷한 문장은 1, 다른 문장은 0으로 레이블링
# 손실 함수는 대조 손실로 설정
from sentence_transformers.losses import ContrastiveLoss
train_loss = ContrastiveLoss(model=model)

In [12]:
# 훈련
model.fit([(train_dataloader, train_loss)], show_progress_bar=True)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5834 [00:00<?, ?it/s]

In [13]:
from sentence_transformers.evaluation import BinaryClassificationEvaluator
eval = BinaryClassificationEvaluator.from_input_examples(eval_examples)
eval.compute_metrices(model)

{'cossim': {'accuracy': 0.8458574181117534,
  'accuracy_threshold': 0.8087599277496338,
  'f1': 0.8297872340425532,
  'f1_threshold': 0.8087599277496338,
  'precision': 0.78,
  'recall': 0.8863636363636364,
  'ap': 0.8713960524696958},
 'manhattan': {'accuracy': 0.8439306358381503,
  'accuracy_threshold': 298.148681640625,
  'f1': 0.8264462809917356,
  'f1_threshold': 322.27545166015625,
  'precision': 0.7575757575757576,
  'recall': 0.9090909090909091,
  'ap': 0.8658555287821843},
 'euclidean': {'accuracy': 0.8458574181117534,
  'accuracy_threshold': 13.242820739746094,
  'f1': 0.8237885462555067,
  'f1_threshold': 13.242820739746094,
  'precision': 0.7991452991452992,
  'recall': 0.85,
  'ap': 0.8677854057431185},
 'dot': {'accuracy': 0.8111753371868978,
  'accuracy_threshold': 416.35687255859375,
  'f1': 0.7924528301886793,
  'f1_threshold': 358.2525634765625,
  'precision': 0.6774193548387096,
  'recall': 0.9545454545454546,
  'ap': 0.8499426891250944}}

In [14]:
# 사전 학습된 모델 평가하기
pretrained_model = SentenceTransformer('jhgan/ko-sroberta-nli')
eval.compute_metrices(pretrained_model)

{'cossim': {'accuracy': 0.8323699421965318,
  'accuracy_threshold': 0.7912960648536682,
  'f1': 0.8160676532769556,
  'f1_threshold': 0.7912960648536682,
  'precision': 0.7628458498023716,
  'recall': 0.8772727272727273,
  'ap': 0.843465397046822},
 'manhattan': {'accuracy': 0.8323699421965318,
  'accuracy_threshold': 160.75051879882812,
  'f1': 0.8176100628930818,
  'f1_threshold': 161.5776824951172,
  'precision': 0.7587548638132295,
  'recall': 0.8863636363636364,
  'ap': 0.8386150988954901},
 'euclidean': {'accuracy': 0.8265895953757225,
  'accuracy_threshold': 7.275073051452637,
  'f1': 0.810924369747899,
  'f1_threshold': 7.299892425537109,
  'precision': 0.75390625,
  'recall': 0.8772727272727273,
  'ap': 0.8380055205019963},
 'dot': {'accuracy': 0.8188824662813102,
  'accuracy_threshold': 102.32652282714844,
  'f1': 0.8041237113402062,
  'f1_threshold': 96.86183166503906,
  'precision': 0.7358490566037735,
  'recall': 0.8863636363636364,
  'ap': 0.8447704663582504}}

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(model.encode(['이 영화 좋아요', '긍정', '부정']))